In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from constants_and_utils import *
from sc_experiments import *

In [3]:
import http.cookiejar as cookielib
import pandas as pd
import requests
import warnings
import logging
import sqlite3
import boto3
import glob
import json
import time
import math
import tqdm
import sys
import os

sys.path.append("/opt/libs")
from apiclass import APIClass,RedshiftClass
from apikeyclass import APIkeyClass
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

In [4]:
rs = RedshiftClass('zhiyin','Zhiyin123')

Successful connection！
 


In [9]:
# before any queries
orig_mem = check_memory_usage()

Total memory: 61.7G; free memory: 45.1G; available memory 50.9G; available memory 82.582%


In [10]:
# how big is index_hs6
before_mem = check_memory_usage()
index = rs.query_df("select * from index_hs6;")
print(len(index))
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
# need to use abs since bytes2human can't simplify negative bytes
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 54.7G; available memory 57.5G; available memory 93.159%
304537
Total memory: 61.7G; free memory: 54.4G; available memory 57.2G; available memory 92.720%
Change in available memory: -277.5M


In [7]:
# how big is logistic data: 10^5 rows
before_mem = check_memory_usage()
df = rs.query_df("select * from logistic_data limit 100000;")
print(len(df))
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 46.7G; available memory 49.1G; available memory 79.510%
100000
Total memory: 61.7G; free memory: 46.6G; available memory 48.9G; available memory 79.248%
Change in available memory: -165.7M


In [13]:
# how big is logistic data: 10^5 rows, selected columns
before_mem = check_memory_usage()
df = rs.query_df(f"select date, supplier_t, buyer_t, hs_code, quantity, amount from logistic_data limit 100000;")
print(len(df))
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 54.7G; available memory 57.5G; available memory 93.130%
100000
Total memory: 61.7G; free memory: 54.6G; available memory 57.4G; available memory 93.054%
Change in available memory: -47.9M


In [8]:
# how big is logistic data: 10^6 rows
before_mem = check_memory_usage()
df = rs.query_df("select * from logistic_data limit 1000000;")
print(len(df))
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 46.6G; available memory 48.9G; available memory 79.249%
1000000
Total memory: 61.7G; free memory: 44.9G; available memory 47.3G; available memory 76.589%
Change in available memory: -1.6G


In [14]:
# how big is logistic data: one-month rows, selected columns
before_mem = check_memory_usage()
df = rs.query_df(f"select date, supplier_t, buyer_t, hs_code, quantity, amount from logistic_data where date like '2022-06%';")
print(len(df))
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 54.6G; available memory 57.4G; available memory 93.053%
2363475
Total memory: 61.7G; free memory: 53.5G; available memory 56.3G; available memory 91.266%
Change in available memory: -1.1G


In [9]:
# try reset df to new variable - this works to deallocate RAM
before_mem = check_memory_usage()
df = "hello"
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
# gain in memory is larger than reduction in memory since in previous cell, we were also deallocating from original df (10^5 rows)
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 44.9G; available memory 47.3G; available memory 76.589%
Total memory: 61.7G; free memory: 46.7G; available memory 49.1G; available memory 79.515%
Change in available memory: +1.8G


In [10]:
# try deleting df - no real difference
before_mem = check_memory_usage()
del df
after_mem = check_memory_usage()
diff = after_mem-before_mem
sign = '-' if after_mem < before_mem else '+'
print('Change in available memory: %s%s' % (sign, bytes2human(abs(diff))))  

Total memory: 61.7G; free memory: 46.7G; available memory 49.1G; available memory 79.515%
Total memory: 61.7G; free memory: 46.7G; available memory 49.1G; available memory 79.515%
Change in available memory: +16.0K
